In [2]:
%pip install matplotlib

  Using cached matplotlib-3.9.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.0-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.54.1-cp312-cp312-win_amd64.whl.metadata (167 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.9.2-cp312-cp312-win_amd64.whl (7.8 MB)
Using cached contourpy-1.3.0-cp312-cp312-win_amd64.whl (218 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.54.1-cp312-cp312-win_amd64.whl (2.2 MB)
Using cached kiwisolver-1.4.7-cp312-cp312-win_amd64.whl (55 kB)
Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Crear generadores de datos para entrenamiento y validación
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)  # Escalar los valores de los píxeles

train_generator = train_datagen.flow_from_directory(
    './PetImages',                 # Directorio con los datos
    target_size=(128, 128),         # Redimensionar imágenes a 128x128 píxeles
    batch_size=32,
    class_mode='binary',     
    classes=['Cat', 'Dog'],        # Clasificación binaria (gato vs perro)
    subset='training'               # Usar el 80% de las imágenes para entrenamiento
)

validation_generator = train_datagen.flow_from_directory(
    './PetImages',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    classes=['Cat', 'Dog'], 
    subset='validation'             # Usar el 20% de las imágenes para validación
)


Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [ ]:
from PIL import Image
import os

# Ruta de tu directorio de imágenes
data_dir = './PetImages'

# Verificar cada archivo en las carpetas de 'Cat' y 'Dog'
for class_name in ['Cat', 'Dog']:
    folder_path = os.path.join(data_dir, class_name)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # Intentar abrir la imagen
            img = Image.open(file_path)
            img.verify()  # Verificar si la imagen está corrupta
            img.close()   # Cerrar la imagen para liberar memoria
        except (IOError, SyntaxError, Image.DecompressionBombError) as e:
            print(f"Error en archivo: {file_path} - {e}")


Error en archivo: ./PetImages\Cat\666.jpg - cannot identify image file 'C:\\Users\\alarc\\Documents\\github\\IA\\img-text\\PetImages\\Cat\\666.jpg'
Error en archivo: ./PetImages\Dog\11702.jpg - cannot identify image file 'C:\\Users\\alarc\\Documents\\github\\IA\\img-text\\PetImages\\Dog\\11702.jpg'


c:\Users\alarc\Documents\github\IA\venv\Lib\site-packages\PIL\TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Definir el modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Salida con una unidad y activación sigmoid para clasificación binaria
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 122s 193ms/step - accuracy: 0.5821 - loss: 0.7005 - val_accuracy: 0.7185 - val_loss: 0.5500
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 122s 196ms/step - accuracy: 0.7355 - loss: 0.5312 - val_accuracy: 0.7639 - val_loss: 0.4902
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 123s 196ms/step - accuracy: 0.7983 - loss: 0.4345 - val_accuracy: 0.8219 - val_loss: 0.3981
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 128s 205ms/step - accuracy: 0.8462 - loss: 0.3511 - val_accuracy: 0.8427 - val_loss: 0.3748
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 124s 198ms/step - accuracy: 0.8776 - loss: 0.2845 - val_accuracy: 0.8407 - val_loss: 0.3696
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 120s 192ms/step - accuracy: 0.9135 - loss: 0.2115 - val_accuracy: 0.8415 - val_loss: 0.3703
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 120s 192ms/step - accuracy: 0.9495 - loss: 0.1338 - val_accuracy: 0.8513 - val_loss: 0.4457
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 120s 192ms/step - accuracy: 0.9686 -

In [17]:

loss, accuracy = model.evaluate(validation_generator)
print(f"Loss: {loss}, Accuracy: {accuracy}")
# Guardar el modelo
model.save('model-cat-dog.keras')


157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 0.8348 - loss: 0.6184
Loss: 0.6031917929649353, Accuracy: 0.8439375758171082


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Cargar una imagen y procesarla
img_path = './dog2.jpg'  # Cambia por la ruta a tu imagen
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img) / 255.0  # Normalizar como en entrenamiento
img_array = np.expand_dims(img_array, axis=0)  # Agregar dimensión de lote

# Hacer la predicción
prediction = model.predict(img_array)
if prediction[0] > 0.5:
    print("Es un perro")
else:
    print("Es un gato")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Es un gato
